In [1]:
#!pip install ray[train]

In [2]:
#!pip install -U ipywidgets
# not needed for production, only for jupyter notebook

In [3]:
#--- should clean imports - copy paste
import ray
from ray.train import ScalingConfig
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor, Normalize, Compose
import ray.train.torch
import ray.util.state
import ray
import torch
import torch.nn as nn
import ray.train.torch
import ray.train
from torch.nn import CrossEntropyLoss, MSELoss
from torch.optim import Adam

In [4]:
ray.init()

2025-01-21 20:33:23,012	WARNING services.py:1996 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=8.07gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2025-01-21 20:33:24,148	INFO worker.py:1743 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.8.10
Ray version:,2.10.0
Dashboard:,http://127.0.0.1:8265


In [8]:
from quickannotator.dl.training import train_model

#@ray.remote
class DLActor:
    def __init__(self,classid,status=-1):
        self.classid=classid
        self.status=status
        
    def start_dlproc(self,is_train=True,allow_pred=True, tile_size=2_048):
        #need to check if is already training, if yes, no opt
        print("starting up")
        
        
        
        #scaling_config = ray.train.ScalingConfig(num_workers=2, use_gpu=True,resources_per_worker={"GPU":.5})
        ## will fail on single node single gpu --- (DLActor pid=174521) Duplicate GPU detected : rank 1 and rank 0 both on CUDA device 1000

        #--- NEED A GPU CONTAINER TO TEST THIS        
        # total_gpus  = ray.cluster_resources().get("GPU", 0)
        # total_gpus_available=  ray.available_resources().get("GPU", 0)
        # print(total_gpus,total_gpus_available)
        # resource_to_request = (total_gpus_available/total_gpus)/2 * 1.01
        # print(f"resource_to_request: {resource_to_request}")

        # scaling_config = ray.train.ScalingConfig(num_workers=total_gpus, use_gpu=True, resources_per_worker={"GPU":resource_to_request})
        

        #----
        scaling_config = ray.train.ScalingConfig(num_workers=1, use_gpu=False)
        trainer = ray.train.torch.TorchTrainer(train_model,
                                       scaling_config=scaling_config,
                                       train_loop_config={'is_train':is_train, 
                                                          'allow_pred':allow_pred,
                                                         'classid':self.classid,
                                                         'tile_size':tile_size})
        self.status = trainer.fit().hex() ##widly -- this doesn't save the result in the actor
        return self.status
    
    def infer(self,image_id,tileids,status=1): #only bulk should be performed, there is no difference between doing 1 versus 100 tiles, so this reduces code complexity
        import sqlalchemy, datetime
        from quickannotator.dl.database import create_db_engine, get_database_path, get_session_aj
        from quickannotator.db import db, Project, Image, AnnotationClass, Notification, Tile, Setting, Annotation, SearchCache

        #open question --- seems like we're getting a lot of seessions, should be a factory process somewhere which also has a pool of existing sessions for reuse


        # probably need to *Delete* the tiles that are associated with this tileid before executing the below, so that they're not duplicated
        # i would do that in a seperate function as "infer" doesn't logically mean "delete", so the expected behavior might be weird if it delets stuff unprompted
        # perhaps that could be clarified with a e.g., named function paramter - or perhaps a seperate function is really needed
        session = get_session_aj(create_db_engine(get_database_path()))

                #id should be modified one we have https://github.com/choosehappy/QuickAnnotator/pull/58
                #imageid will need to be included - end result is a multikey -(imageid, tileid, classid)
        stmt = sqlalchemy.update(Tile).where(Tile.id.in_(tileids), Tile.image_id == image_id,
                                              Tile.annotation_class_id == self.classid)\
                                                .values(seen=status) #  ,date=datetime.datetime.now())) ## should add date time

        # Execute the update
        session.execute(stmt)
        session.commit()
        
        # Close the session
        session.close()


        #need a similar statement to get the DL starting
        return True
    

    def getTileStatus(self,tileids): #probably belongs else where but need this for debug
        import sqlalchemy, datetime
        from quickannotator.dl.database import create_db_engine, get_database_path, get_session_aj
        from quickannotator.db import db, Project, Image, AnnotationClass, Notification, Tile, Setting, Annotation, SearchCache

        
        session = get_session_aj(create_db_engine(get_database_path()))

        stmt = session.query(Tile).filter(Tile.id.in_(tileids),
                          Tile.annotation_class_id == self.classid)

        # Execute the query
        result = stmt.all()
        
        # Close the session
        session.close()

        return result


    def getclassid(self):
        return self.classid
        
    
    def getstatus(self):
        print("in get status!")
        print(f"self.status:\t {self.status}")
        return self.status

    def setstatus(self,status):
        self.status=status
        return self.status



In [9]:
##not ray test
class2=DLActor(classid=2)

#class2.start_dlproc(is_train=True,allow_pred=True)

In [12]:
class2.infer(image_id=1,tileids=[323],status=1)

InitSpatiaMetaData() error:"table spatial_ref_sys already exists"


True

In [6]:
class2 = DLActor.options(max_concurrency=2).remote(classid=2)


In [ ]:
class2.bulk


In [ ]:
print(ray.get(class2.getclassid.remote()))


In [ ]:
ref2=class2.start_dlproc.remote()
print(ref2)

In [18]:
%load_ext autoreload

In [19]:
%autoreload 2

In [ ]:
image_id = 1
tile_id = 323
class2.infer.remote([tile_id])

In [20]:
from quickannotator.dl.training import train_model

In [33]:
from quickannotator.db import db, Project, Image, AnnotationClass, Notification, Tile, Setting, Annotation, SearchCache


In [ ]:
#--- need to run 
#memcached -m 10240 -I 20m -vv -u www-data &

In [ ]:
train_model({"classid":2,"tile_size":2_048})